# Limpieza del texto y categorizacion

In [1]:
# IMPORTS
import pandas as pd
import os, re

In [2]:
# RUTAS 
BASE_PATH = r"C:\Users\usuario\OneDrive - POTENCIA\ARCHIVOS\TAREA_ENTIDADES"
FILTERPATH = os.path.join(BASE_PATH, "data", "2_categorized")
TRYPATH = os.path.join(BASE_PATH, "data", "99_try")
os.makedirs(FILTERPATH, exist_ok=True)
os.makedirs(TRYPATH, exist_ok=True)

# Data
df = pd.read_excel("../data/0_raw/SECOP_filtro__2019_2025.xlsx")

print(f"📍 DataFrame Inicial:\n Dimensiones: {df.shape}")

📍 DataFrame Inicial:
 Dimensiones: (7779, 28)


In [3]:
# LIMPIEZA DE TEXTO 

# Diccionario de palabras a omitir al inicio de los textos
init_words = [
        'el', 'la', 'las', 'los', 'a', 'de', 'del', 'para', 
        'bs', 'c', 'e', 'es', 'sg', 'srt', 'mr', 'd', 'dt', 'nsa',
        's', 'sa', 'se', 
        'lote', 'rmtc', 'rstc', 'rtvc', 'rvlc', 'no',
        'spa', 'oap', 'sol', 'nr', 'ce', 'srn', 'srnc',
        'ranc', 'ratc', 'rcnc', 'fun', 'fortis', 
        'amf', 'amfis', 'sm', 'dtnsa', 
        'realizar', 'rrealizar', 'realizacion', 'ealizar',
        'actividades', 'grupo', 'servicio', 'servicios',
        'ejecucion', 'ejecutar', 'esfuerzos', 
        'prestar', 'global', 'segunda', 'fase',
        'obra', 'obras', 'publica', 'civil', 'civiles', 'complementarias',
        'llevar a cabo', 'mano', 'necesarias', 'segunda',
        'mediante', 'por', 'sistema', 'sin', 'formula', 
        'ajuste', 'reajuste', 'todo', 'costo',
        'aunar', 'anuar', 'unar', 'esfuerzo', 'y' , 'apoyo mutuo', 
        'precios', 'precio', 'unitario', 'unitarios', 'fijo', 'fijos',
        'contratar', 'bajo', 
]

In [4]:
# FUNCIÓN - limpiar palabras iniciales
def limpiar_inicio(sentence, palabras_ini):
    if not isinstance(sentence, str):
        return sentence
    
    sentence = sentence.strip()
    # eliminar todas las palabras iniciales consecutivas que estén en la lista
    while True:
        partes = sentence.split(maxsplit=1)
        if partes and partes[0].lower() in palabras_ini:
            sentence = partes[1] if len(partes) > 1 else ""
        else:
            break
    return sentence

df['texto'] = df['texto'].apply(lambda x: limpiar_inicio(x, init_words))
conteo_1 = df['texto'].value_counts()
print(f"Coincidencias {conteo_1.shape}")
out = os.path.join(TRYPATH, 'Limpieza1.xlsx')
conteo_1.to_excel(out) 

Coincidencias (6707,)


In [5]:
# FUNCION - Categorización categorias y categoria principal

def clasificar(texto, sector):
    if not isinstance(texto, str):
        return {"todas": "Otros", "principal": "Otros"}

    texto = texto.lower()
    encontradas = []
    posiciones = {}

    for cat, palabras in sector.items():
        for palabra in palabras:
            # Coincidencia exacta de palabra o frase
            match = re.search(rf"\b{re.escape(palabra)}\b", texto)
            if match:
                encontradas.append(cat)
                pos = match.start()
                if cat not in posiciones or pos < posiciones[cat]:
                    posiciones[cat] = pos
                break

    if not encontradas:
        return {"todas": "Otros", "principal": "Otros"}

    todas = "; ".join(encontradas)
    principal = min(posiciones, key=posiciones.get)

    return {"todas": todas, "principal": principal}

In [6]:
# clasificacion - Por objeto contractual
objeto = {
    'Adecuacion': {'adecuacion', 'adecuar', 'adecuaciones', 'acondicionamiento', 'habilitar'},
    'Construccion': {'construccion', 'construir', 'construcciones', 'reconstruir', 'reconstruccion',
                     'demoler', 'demolicion', 'desmontaje', 'desmonte', 'desmontar', 'instalacion'},
    'Mantenimiento': {'mantenimiento', 'mantener'},
    'Reparacion': {'reparacion', 'reparaciones', 'reparar', 'rehabilitar', 'recuperacion', 'rehabilitacion', 'restauracion'},
    'Atencion': {'atencion', 'atender'},
    'Mejoramiento': {'mejoramiento', 'mejorar', 'remodelar', 'remodelacion', 'ampliar', 'ampliacion', 'modernizacion'},
}


# Aplicar a una columna del DataFrame
df[["objetos", "objeto_contractual"]] = df["texto"].apply(
    lambda x: pd.Series(clasificar(x, objeto))
)

print("Tabla de conteo por objeto contractual:")
tabla = (df["objeto_contractual"].value_counts(dropna=False).rename("conteo").reset_index())
tabla["porcentaje"] = (tabla["conteo"] / tabla["conteo"].sum() * 100).round(2)
tabla.columns = ["objeto_contractual", "valor_neto", "porcentaje"]
print(tabla)

Tabla de conteo por objeto contractual:
  objeto_contractual  valor_neto  porcentaje
0      Mantenimiento        3965       50.97
1         Adecuacion        1186       15.25
2       Mejoramiento         957       12.30
3       Construccion         883       11.35
4              Otros         466        5.99
5         Reparacion         225        2.89
6           Atencion          97        1.25


In [19]:
# sub categorias 
dic_subcategorias = {
    'Rio': {
            'rio', 'cuenca', 'cuenca hidrografica', 'canal', 'canal hidraulico', 'afluente', 
            'corriente hidrica', 
            'control de inundaciones', 'proteccion de rivera', 'obras hidraulicas'
            },

    'Reservas y ecoparques': {
            'reserva natural', 'area protegida', 'ecoparque', 'restauracion ambiental', 
            'reforestacion','biodiversidad', 'conservacion ambiental','ecosistema', 
            'gestion ambiental','zona costera', 'manejo ambiental', 'ecologico', 
            'sendero', 'ecoturistico', 'natural'
            },
    'Energia renovable': {
            'energia renovable', 'energia solar', 'panel solar', 'sistema fotovoltaico',
            'energia eolica', 'generacion electrica limpia'
        },
    'Servicios publicos': {
            'servicio publico', 'servicios publicos', 'domiciliario', 'acueducto', 'alcantarillado', 
            'tratamiento de aguas', 'agua potable', 'residuos solidos', 'aseo urbano', 
            'disposicion final', 'gas domiciliario'
        },
    'Agro': {
            'agropecuario', 'agricola','ganaderia', 'desarrollo rural',
            'sistema de riego', 'distrito de riego','asistencia tecnica rural'
        },
    'Turismo': {
            'turismo', 'infraestructura turistica','atractivo turistico', 'ecoturismo',
            'turismo cultural', 'ruta turistica'
        }, 
   
    'Aeropuerto': {
            'aeropuerto', 'infraestructura aeroportuaria', 'terminal aereo', 'aviacion civil'
        },
    'Puente': {
            'puente vehicular', 'puente peatonal', 'paso elevado', 'interseccion vial','rotonda vehicular', 'puente'
        },
    'Puerto': {
            'puerto', 'infraestructura portuaria', 'muelle', 'embarcadero', 
            'terminal fluvial', 'navegacion fluvial'
        },
    'Transporte publico': {
            'transporte publico', 'movilidad urbana','sistema de transporte masivo',
            'terminal de transporte', 'bus','metro', 'cicloruta', 'bicicarril'
        },
    'Tren': {
            'tren', 'ferrocarril', 'infraestructura ferroviaria', 'red ferroviaria'
        },
    'Vias': {
            'vias', 'via', 'via nacional', 'carretera', 'red vial', 'corredor vial', 'pavimentacion', 
            'mejoramiento vial', 'glorieta', 'interseccion vial'
        },
    'Vias terciarias': {
            'via terciaria', 'red vial terciaria', 'camino rural', 'mejoramiento de vias rurales'
        },
    
    'Parques y plazas': {
            'parque urbano', 'plaza publica', 'espacio publico', 'zona recreativa',
            'escenario recreativo', 'plazoleta'
        },
    'Vias urbanas': {
            'via urbana', 'infraestructura urbana', 'anden', 'andenes',
            'malla vial urbana', 'pavimentacion urbana'
        },
    'Vivienda': {
            'vivienda', 'proyecto habitacional', 'mejoramiento de vivienda',
            'urbanizacion', 'solucion de vivienda'
        },
    'Educacion': {
            'educacion', 'institucion educativa', 'colegio', 'escuela', 'universidad', 
            'infraestructura educativa', 'aulas', 'sede educativa'
        },
    'Deporte': {
            'deporte', 'escenario deportivo', 'polideportivo', 'coliseo',
            'cancha deportiva', 'unidad deportiva'
    },

    'Publico': {
        'batallon', 'batallones', 'infanteria', 'estacion de policia', 'policia',
        'aerocivil', 'insituto nacional de medicina legal', 'oficinas', 
        'base naval', 'palacio de justucia', 'juzgado', 'superintendencia', 
        'militar', 'militares', 'consejo', 'inpec', 'pabellon', 'pabellones'
        'hospital', 'centro de salud', 'eps', 'clinica', 'salud'
    }
}

df[["sub_categorias", "SUB"]] = df["texto"].apply(
    lambda x: pd.Series(clasificar(x, dic_subcategorias))
)

print("Tabla de conteo por SUB-categorias:")
tabla_2 = (df["SUB"].value_counts(dropna=False).rename("conteo").reset_index())
tabla_2["porcentaje"] = (tabla_2["conteo"] / tabla_2["conteo"].sum() * 100).round(2)
tabla_2.columns = ["SUB", "conteo", "porcentaje"]
print(tabla_2)

Tabla de conteo por SUB-categorias:
                      SUB  conteo  porcentaje
0                   Otros    3051       39.22
1                    Vias    1723       22.15
2                 Publico    1158       14.89
3              Aeropuerto     481        6.18
4               Educacion     319        4.10
5         Vias terciarias     303        3.90
6      Servicios publicos     222        2.85
7                  Puerto     127        1.63
8                  Puente      71        0.91
9                 Deporte      64        0.82
10                    Rio      56        0.72
11               Vivienda      55        0.71
12                   Agro      54        0.69
13  Reservas y ecoparques      45        0.58
14       Parques y plazas      19        0.24
15                Turismo      14        0.18
16           Vias urbanas      13        0.17
17      Energia renovable       3        0.04
18     Transporte publico       1        0.01


In [20]:
# Clases no relveantes

# Seleccionar solo otros
otros = df[df["SUB"] == "Otros"].copy()
publico = df[df["SUB"] == "Publico"].copy()

# Exportar resultados
otros.to_excel(os.path.join(TRYPATH, 'otros.xlsx'), index=False)
publico.to_excel(os.path.join(TRYPATH, 'publico.xlsx'), index=False)

In [22]:
dic_macro = {
    'Ambiental y gestion del territorio': {
        'Rio',
        'Reservas y ecoparques'
    },
    'Productiva y de servicios': {
        'Energia renovable',
        'Servicios publicos',
        'Agro',
        'Turismo'
    },
    'Transporte': {
        'Aeropuerto',
        'Puente',
        'Puerto',
        'Transporte publico',
        'Tren',
        'Vias',
        'Vias terciarias'
    },
    'Urbanismo y desarrollo metropolitano': {
        'Parques y plazas',
        'Vias urbanas',
        'Vivienda',
        'Educacion',
        'Deporte'
    },
    'Publico': {
        'Publico'
    },
    'Otros': {
        'Otros' 
    }
}
df['MACRO'] = df['SUB'].map({sub: macro for macro, subs in dic_macro.items() for sub in subs})
contratos = df[~df['MACRO'].isin(['Otros', 'Publico'])]
print(f"📍 DataFrame filtrado por MACRO categorias:\n Dimensiones: {contratos.shape}")
print(f"Representa: {round((contratos.shape[0] / df.shape[0] * 100),2)}%")

print("Tabla de conteo por MACRO categorias proyecto:")
tabla = (contratos["MACRO"].value_counts(dropna=False).rename("conteo").reset_index())
tabla["porcentaje"] = (tabla["conteo"] / tabla["conteo"].sum() * 100).round(2)
tabla.columns = ["MACRO", "conteo", "porcentaje"]
print(tabla)


📍 DataFrame filtrado por MACRO categorias:
 Dimensiones: (3570, 33)
Representa: 45.89%
Tabla de conteo por MACRO categorias proyecto:
                                  MACRO  conteo  porcentaje
0                            Transporte    2706       75.80
1  Urbanismo y desarrollo metropolitano     470       13.17
2             Productiva y de servicios     293        8.21
3    Ambiental y gestion del territorio     101        2.83


In [23]:
# Exportar resultados
contratos.to_excel(os.path.join(FILTERPATH, 'SECOP_CAT_final.xlsx'), index=False)


____________________